![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

# LLM Session Memory - Multiple Sessions

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.
This notebook will show how to use Redis to structure and store and retrieve this conversational session memory and how to manage multiple sessions simultaneously.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/session-manager/01_multiple_sessions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Environment setup

In [ ]:
%pip install cohere "redisvl>=0.4.1" sentence-transformers

### Set Cohere API Key

In [ ]:
import os
import getpass


if "COHERE_API_KEY" not in os.environ:
    os.environ["COHERE_API_KEY"] = getpass.getpass("COHERE_API_KEY")

#### Run local redis (for colab)
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL and Client

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [ ]:
import os
from redis import Redis

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

redis_client = Redis.from_url(REDIS_URL)

#### Initialize CohereClient as LLM layer
To demonstrate how a real LLM conversation may flow we'll use a real LLM.

In [ ]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self.remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text

    def remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "user":
                new_statement = {"role": "USER", "message": statement["content"]}
            elif statement["role"] == "llm":
                new_statement = {"role": "CHATBOT", "message": statement["content"]}
            elif statement["role"] == "system":
                new_statement = {"role": "SYSTEM", "message": statement["content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

client = CohereClient()

### Import SemanticSessionManager

redisvl provides the SemanticSessionManager for easy management of session state.
It also allows for tagging of messages to separate conversation sessions with the `session_tag` optional parameter.
Let's create a few personas that can talk to our AI.


In [ ]:
student = 'student'
yp = 'young professional'
retired = 'retired pensioner'

In [ ]:
from redisvl.extensions.session_manager import SemanticSessionManager

session = SemanticSessionManager(name='budgeting help')

#### Here we'll have multiple separate conversations simultaneously, all using the same session manager.
#### Let's add some conversation history to get started.

#### We'll assign each message to one of our users with their own `session_tag`.

In [14]:
# adding messages to the student session
session.add_messages(
    [{"role":"system",
      "content":"You are a personal assistant helping people create sound financial budgets. Be very brief and concise in your responses."},
     {"role":"user",
      "content":"I'm a college student living in Montana and I need help creating a budget. I am a first year accounting student."},
     {"role":"llm",
      "content":"Sure, I can help you with that. What is your monthly income and average monthly expenses?"},
     {"role":"user",
      "content":"my rent is $500, utilities are $100, and I spend $200 on groceries. I make $1000 a month as a part time tutor."},
     ],
    session_tag=student)

#adding messages to the young professional session
session.add_messages(
    [{"role":"system",
      "content":"You are a personal assistant helping people create sound financial budgets. Be very brief and concise in your responses."},
     {"role":"user",
      "content":"I'm a young professional living in New York City and I need help planning for retirement. I already have a sizable emergency fund."},
     {"role":"llm",
      "content":"Sure I can help you with that. What is your monthly income and average monthly expenses?"},
     {"role":"user",
      "content":"I make $5000 a month as a software engineer. My rent is $2000, utilities are $200, groceries are $300, and I spend $500 on entertainment."},
     ],
    session_tag=yp)

#adding messages to the retiree session
session.add_messages(
    [{"role":"system",
      "content":"You are a personal assistant helping people create sound financial budgets. Be very brief and concise in your responses."},
        {"role":"user",
        "content":"I'm a retired pensioner living in Florida and I need help creating a budget."},
        {"role":"llm",
        "content":"Sure I can help you with that. What is your monthly income and average monthly expenses?"},
        {"role":"user",
        "content":"I make $2000 a month from my pension. I own my home outright, utilities are $100, groceries are $200, and I spend $100 on entertainment."},
     ],
    session_tag=retired)

#### With the same session manager calling the same LLM we can handle distinct conversations. There's no need to instantiate separate classes or clients.

#### Just retrieve the conversation of interest using the same `session_tag` parameter when fetching context.

In [15]:
prompt = "What is the single most important thing I should focus on financially?"
context = session.get_recent(session_tag=student)
response = client.converse(prompt=prompt, context=context)
session.store(prompt, response, session_tag=student)
print('Student: ', prompt)
print('\nLLM: ', response)

Student:  What is the single most important thing I should focus on financially?

LLM:  Focus on keeping expenses low.


In [16]:
prompt = "What is the single most important thing I should focus on financially?"
context = session.get_recent(session_tag=yp)
response = client.converse(prompt=prompt, context=context)
session.store(prompt, response, session_tag=yp)
print('Young Professional: ', prompt)
print('\nLLM: ', response)

Young Professional:  What is the single most important thing I should focus on financially?

LLM:  Max out your 401(k) contributions to take advantage of compound interest and any employer matching.


In [17]:
prompt = "What is the single most important thing I should focus on financially?"
context = session.get_recent(session_tag=retired)
response = client.converse(prompt=prompt, context=context)
session.store(prompt, response, session_tag=retired)
print('Retiree: ', prompt)
print('\nLLM: ', response)

Retiree:  What is the single most important thing I should focus on financially?

LLM:  With your current income and expenses, your focus should be on maintaining your current financial situation and ensuring your long-term financial stability. Review your budget regularly and adjust as necessary.


#### You can see how each conversation is stored separately.

In [18]:
for ctx in session.get_recent(session_tag=student):
    print(ctx)

{'role': 'user', 'content': "I'm a college student living in Montana and I need help creating a budget. I am a first year accounting student."}
{'role': 'llm', 'content': 'Sure, I can help you with that. What is your monthly income and average monthly expenses?'}
{'role': 'user', 'content': 'my rent is $500, utilities are $100, and I spend $200 on groceries. I make $1000 a month as a part time tutor.'}
{'role': 'user', 'content': 'What is the single most important thing I should focus on financially?'}
{'role': 'llm', 'content': 'Focus on keeping expenses low.'}


In [13]:
session.clear()